In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import time 
import random
import json
import torch
from torch.nn.utils.rnn import pack_sequence
from torch.utils.data import DataLoader,Dataset

/Users/hanliying/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#for colab
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My Drive/Pylon/test_examples/full_labeled_data

### Process Raw Data to Generate Primitive Activitiy Dataset
The data we use here is from the Third Nurse Care Activity Recognition Challenge. It contains 3-dimensional accelerometer data from 12 subjects performing 27 different activities. We have filtered out noisy data and stored them in 'data_full.json', but further preprocessing is necessary since each activity data has arbitrary length. We are going to segment each data with the same lenghth, and choose the activity classes with balanced training data size to be the primitive activities.

In [4]:
'''Read file and convert string to Python dict'''

with open('./complex_event/data_full.json','r') as f:
  load_dict = json.load(f)
  # print(load_dict)


In [5]:
'''Segment the raw data into same time length'''

SEGMENT_SIZE = 60
SLIDING_SIZE = 5

'''Choose primitive activitiy class indexed by target_label'''

target_label = [4, 2, 12, 9]
label_map = {4:0, 2:1, 12:2, 9:3}

n_class = len(target_label)
n_data_per_class = 2000

In [6]:
'''Generate primitive activity data'''

from collections import Counter

x_prim_data = []
y_prim_data = []
time_period = set()
activity_len = {}
for item in load_dict:
  x, y, start, finish = item['acc'], item['old_label'], item['start'], item['finish']
  counter = dict(Counter(y_prim_data))
  if y in target_label and (start, finish) not in time_period and len(x) >= SEGMENT_SIZE:
    # Check duplicates
    time_period.add((start, finish))
    # Divide y into segements of length SEGMENT_SIZE
    # TODO: Here just discard segments with length smaller than SEGMENT_SIZE, we can use imputation later to keep them
    num = int((len(x) - SEGMENT_SIZE)/SLIDING_SIZE) + 1
    for i in range(num):
      x_i = x[i * SLIDING_SIZE:i * SLIDING_SIZE + SEGMENT_SIZE]
      # y_i = y
      y_i = label_map[y]
      x_prim_data.append(x_i)
      y_prim_data.append(y_i)
      
      # count the number of data with different length for each activity
      length = len(x_i)
      if y not in activity_len:
        activity_len[y] = [length]
      else:
        activity_len[y].append(length)
      


print(len(x_prim_data))
result = dict(Counter(y_prim_data))
print(result)


# count the number of data with different length for each activity
for key, val in activity_len.items():
  n = Counter(val)
  print("Activity",key,":")
  print("The range of data length is (", min(val), ",", max(val), ")")
  print("The distribution of data length is", n)

temp = list(zip(x_prim_data, y_prim_data))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
x_prim_data, y_prim_data = list(res1), list(res2)

x_prim_data = np.array(x_prim_data).transpose((0,2,1))
x_prim_data = x_prim_data.astype(np.float32)
y_prim_data = np.array(y_prim_data)

# balance data set of different activities
x_prim_data_temp = np.zeros([n_data_per_class, x_prim_data.shape[1], x_prim_data.shape[2]])
y_prim_data_temp = np.zeros([n_data_per_class, 1])
for i in range(n_class):
  idx = np.where(y_prim_data==i)[0]
  idx = np.random.choice(idx, n_data_per_class, replace=False)
  if i == 0:
    x_prim_data_temp = x_prim_data[idx] 
    y_prim_data_temp = y_prim_data[idx] 
  else:
    x_prim_data_temp = np.concatenate([x_prim_data_temp,x_prim_data[idx]],axis=0)
    y_prim_data_temp = np.concatenate([y_prim_data_temp,y_prim_data[idx]],axis=0)

x_prim_data = x_prim_data_temp
y_prim_data = y_prim_data_temp

# shuffle again
temp = list(zip(x_prim_data, y_prim_data))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
x_prim_data, y_prim_data = list(res1), list(res2)

x_prim_data = np.array(x_prim_data)
x_prim_data.astype(np.float32)
y_prim_data = np.array(y_prim_data)

# split some data to test set
x_prim_test = x_prim_data[-int(n_data_per_class/4*n_class):]
x_prim_train = x_prim_data[:-int(n_data_per_class/4*n_class)]
y_prim_test = y_prim_data[-int(n_data_per_class/4*n_class):]
y_prim_train = y_prim_data[:-int(n_data_per_class/4*n_class)]

print(dict(Counter(y_prim_test)))

print(x_prim_train.shape)
print(y_prim_train.shape)
print(x_prim_test.shape)
print(y_prim_test.shape)

x_prim_train = torch.from_numpy(x_prim_train)
y_prim_train = torch.from_numpy(y_prim_train)
x_prim_test = torch.from_numpy(x_prim_test)
y_prim_test = torch.from_numpy(y_prim_test)

# print(x_prim_train.shape)
# print(y_prim_train.shape)
# print(x_prim_test.shape)
# print(y_prim_test.shape)


171789
{0: 69316, 1: 72562, 3: 11705, 2: 18206}
Activity 4 :
The range of data length is ( 60 , 60 )
The distribution of data length is Counter({60: 69316})
Activity 2 :
The range of data length is ( 60 , 60 )
The distribution of data length is Counter({60: 72562})
Activity 9 :
The range of data length is ( 60 , 60 )
The distribution of data length is Counter({60: 11705})
Activity 12 :
The range of data length is ( 60 , 60 )
The distribution of data length is Counter({60: 18206})
{0: 498, 1: 503, 2: 488, 3: 511}
(6000, 3, 60)
(6000,)
(2000, 3, 60)
(2000,)


In [7]:
print(np.array(x_prim_train).shape)
print(np.array(x_prim_test).shape)

(6000, 3, 60)
(2000, 3, 60)


### Neural Modules to Calssify Primitive Activity

In [8]:
'''Define Neural Architecture'''

from torch.nn.modules.activation import Softmax
from torch.nn.modules import dropout
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CRNN(nn.Module):
    def __init__(self, n_class, drop_out=0.5):
      super().__init__()
      self.n_class = n_class
      self.conv = nn.Sequential(
          nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, padding='same'),
          nn.LeakyReLU(),
          nn.MaxPool1d(kernel_size=3, stride=3),
          nn.BatchNorm1d(num_features=8),
          nn.Dropout(drop_out),

          # nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, padding='same'),
          # nn.LeakyReLU(),
          # nn.MaxPool1d(kernel_size=3, stride=3),
          # nn.BatchNorm1d(num_features=16),
          # nn.Dropout(drop_out)
      )

      self.lstm = nn.LSTM(input_size=20, hidden_size=8, bidirectional=True, batch_first=True)
      self.fc = nn.Sequential(
          nn.LeakyReLU(),
          nn.Dropout(drop_out),
          nn.Flatten(),
          nn.Linear(in_features=128, out_features=self.n_class),
          nn.Softmax(dim=1)
      )
        

    def forward(self, x):
      x = self.conv(x)
      # print(x.shape)
      x, _ = self.lstm(x)
      # print(x.shape)
      x = self.fc(x)
      # print(x.shape)
      return x



In [9]:
'''
def train(model, x_train, y_train, epoch, batch_size):
  batch_num = int(np.ceil(x_train.shape[0] / batch_size))
  for k in range(epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i in range(batch_num):
        # get the inputs; data is a list of [inputs, labels]
        if i == batch_num - 1:
          inputs, labels = x_train[i * batch_size:], y_train[i * batch_size:]
        else:
          inputs, labels = x_train[i * batch_size:(i + 1) * batch_size], y_train[i * batch_size:(i + 1) * batch_size]
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        running_acc += (predicted == labels).sum().item() / batch_size
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[{k + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f} acc: {running_acc / 100:.3f}')
            running_loss = 0.0
            running_acc = 0.0

  print('Finished Training')
'''

"\ndef train(model, x_train, y_train, epoch, batch_size):\n  batch_num = int(np.ceil(x_train.shape[0] / batch_size))\n  for k in range(epoch):  # loop over the dataset multiple times\n    running_loss = 0.0\n    running_acc = 0.0\n    for i in range(batch_num):\n        # get the inputs; data is a list of [inputs, labels]\n        if i == batch_num - 1:\n          inputs, labels = x_train[i * batch_size:], y_train[i * batch_size:]\n        else:\n          inputs, labels = x_train[i * batch_size:(i + 1) * batch_size], y_train[i * batch_size:(i + 1) * batch_size]\n        \n        criterion = nn.CrossEntropyLoss()\n        optimizer = optim.Adam(model.parameters(), lr=0.001)\n\n        # zero the parameter gradients\n        optimizer.zero_grad()\n\n        # forward + backward + optimize\n        outputs = model(inputs)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        # print statistics\n        running_loss += loss.item()\n     

In [10]:
'''Evaluation function used for primitive activity classification'''

def evaluate(model, x_test, y_test):
  correct = 0
  total = y_test.shape[0]
  with torch.no_grad():
    inputs, labels = x_test, y_test
    outputs = model(inputs)
    _, predict = torch.max(outputs.data, 1)
    correct = (predict == labels).sum().item() / total

  print('Accuracy of the network on the test data: %d %%' % (
    100 * correct))
  
  class_correct = list(0. for i in range(model.n_class))
  class_total = list(0. for i in range(model.n_class))
  with torch.no_grad():
    inputs, labels = x_test, y_test
    outputs = model(inputs)
    _, predict = torch.max(outputs.data, 1)
    c = (predict == labels).squeeze()
    for i in range(labels.shape[0]):
      label = labels[i]
      class_correct[label] += c[i].item()
      class_total[label] += 1

  print(class_total)
  for i in range(model.n_class):
    print('Accuracy of activity %2s : %2d %%' % (
    i, 100 * class_correct[i] / class_total[i]))
  return [predict, correct]


### Generate Complex Event Dataset

In [11]:
''' Generate complex event training/test set using prmitive activities '''

n_data_per_event = 6000

''' 
event_dict[key]: complex event class ID.
event_dict[item]: complex event pattern of the corresponding complex event class.

In this simple example, event 0 means "primitive activity 1 happens before 3", event 1 means "primitive activity 0 happens before 2"
'''
event_dict = {0: [(1,0,3), (1,2,3), (1,3,0), (1,3,2)], 1: [(0,1,2), (0,3,2), (0,2,1), (0,2,3)]}


idx_list = []
for label in range(n_class):
    idx = list(np.where(y_prim_data == label)[0])
    idx_list.append(idx)

x_event = np.empty((0,x_prim_data.shape[1],x_prim_data.shape[2]*3))
y_event = np.empty((0))

for event_id, event_rules in event_dict.items():
    n_sub_event = len(event_rules)
    n_data_per_comb = int(n_data_per_event/n_sub_event)

    for i in range(n_sub_event):
        event_rule = event_rules[i]
        e0, e1, e2 = event_rule[0], event_rule[1], event_rule[2]
        x_event_sub = np.concatenate((x_prim_data[np.random.choice(idx_list[e0], n_data_per_comb)],
                                      x_prim_data[np.random.choice(idx_list[e1], n_data_per_comb)],
                                      x_prim_data[np.random.choice(idx_list[e2], n_data_per_comb)]),
                                     axis=2)
        x_event = np.append(x_event, x_event_sub, axis=0)
        
    y_event = np.append(y_event, np.array([event_id] * n_data_per_event))


# shuffle the list
temp = list(zip(x_event, y_event))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
x_event, y_event = np.array(res1), np.array(res2)

# Construct trainig set and test set
x_event = x_event.astype(np.float32)
y_event = y_event.astype(np.int32)

# x_event, y_event = utils.shuffle(x_event, y_event)

n_event_data = len(x_event)

x_event_train = x_event[:-int(n_event_data/5)]
x_event_test = x_event[-int(n_event_data/5):]
y_event_train = y_event[:-int(n_event_data/5)]
y_event_test = y_event[-int(n_event_data/5):]

x_event_train = torch.from_numpy(x_event_train)
y_event_train = torch.from_numpy(y_event_train)
x_event_test = torch.from_numpy(x_event_test)
y_event_test = torch.from_numpy(y_event_test)

print(x_event_train.shape)
print(y_event_train.shape)
print(x_event_test.shape)
print(y_event_test.shape)

print(y_event_train)

torch.Size([9600, 3, 180])
torch.Size([9600])
torch.Size([2400, 3, 180])
torch.Size([2400])
tensor([0, 1, 0,  ..., 0, 1, 0], dtype=torch.int32)


In [12]:
print(np.array(x_event_train).shape)
print(np.array(x_event_test).shape)

(9600, 3, 180)
(2400, 3, 180)


In [13]:
def predict_sequence(model, x_event, segment_size):
    '''
    model: the primitive activity classifer
    x_event: the complex event data of shape (batch_size, num_channel, sequence_length)
    segment_size: the length of the input required by primitive activity classifier
    '''
    segments_num = int(x_event.shape[-1] / segment_size)
    activity_sequence = np.zeros([x_event.shape[0],1])
    prob_sequence = np.zeros([x_event.shape[0],1])
    logits_sequence = []

    for i in range(segments_num):
        x_event_temp = x_event[:,:,i * segment_size:(i + 1) * segment_size]
        # y_event_temp = y_event[i * segment_size:(i + 1) * segment_size]
        # print(x_event_temp.shape)
        outputs = model(x_event_temp)
        # prob, label = torch.max(outputs.data,1)
        # print(label.data.dtype)
        logits_sequence.append(outputs)
        # if i == 0:
            # activity_sequence = label.data.reshape(-1,1)
            # prob_sequence = prob.data.reshape(-1,1)
        # else:
        #     activity_sequence = np.concatenate([activity_sequence, label.data.reshape(-1,1)], axis=1)
        #     prob_sequence = np.concatenate([prob_sequence, prob.data.reshape(-1,1)], axis=1)
    
    return logits_sequence

In [14]:
'''Define constraint function and Pylon constraint loss'''

import sys
sys.path.append("../")

from pylon.constraint import constraint
from pylon.brute_force_solver import SatisfactionBruteForceSolver
# from pylon.brute_force_solver import *

def complex_event(*logits_sequence, **kwargs):
    '''
    logits_sequence: a sequence of logits tensors returned by the primitive classifier
    kwargs['event_label']: ground truth complex event label
    kwargs['event_dict']: a dictionary recording the complex event class and corresponding patterns, event_dict[event_id] = [(event_pattern1), (event_pattern2), ...]
    '''
    for event_id, event_rules in kwargs['event_dict'].items():
        if logits_sequence in event_rules:
            return kwargs['event_label'] == event_id  # (torch,torch,torch) == (int,int,int)
    return False

complex_event_cons = constraint(complex_event, SatisfactionBruteForceSolver())

## Training and Evaluation

In [15]:
model = CRNN(n_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [16]:
predict, acc = evaluate(model, x_prim_test, y_prim_test)

Accuracy of the network on the test data: 26 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 24 %
Accuracy of activity  1 :  5 %
Accuracy of activity  2 : 79 %
Accuracy of activity  3 :  2 %


In [17]:
from tqdm import tqdm

NUM_EPOCHS = 1

for epoch in range(NUM_EPOCHS):
      
    # train
    for i in tqdm(range(len(x_event_train))):
        model.train()
        optimizer.zero_grad()
        # print(len(x_event_train))
        logits_sequence = predict_sequence(model, x_event_train[i].unsqueeze(0), SEGMENT_SIZE)

        closs = complex_event_cons(*logits_sequence, event_label=y_event_train[i], event_dict=event_dict)

        closs.backward()
        optimizer.step()
        
        if i % 1000 == 0 and i != 0:
            evaluate(model, x_prim_test, y_prim_test)
        
    evaluate(model, x_prim_test, y_prim_test)

  0%|          | 0/9600 [00:00<?, ?it/s]/Users/hanliying/Documents/UCLA/Research/Neurosym_proj/pylon-master/examples/../pylon/brute_force_solver.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  else torch.tensor(data=self.cond(*sample,**kwargs), dtype=torch.bool) for sample in samples ])
 10%|█         | 1004/9600 [00:24<03:58, 36.03it/s]

Accuracy of the network on the test data: 24 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 20 %
Accuracy of activity  1 : 22 %
Accuracy of activity  2 : 59 %
Accuracy of activity  3 :  2 %


 21%|██        | 2006/9600 [00:50<03:22, 37.54it/s]

Accuracy of the network on the test data: 26 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 25 %
Accuracy of activity  1 : 46 %
Accuracy of activity  2 : 34 %
Accuracy of activity  3 :  2 %


 31%|███▏      | 3005/9600 [01:13<03:00, 36.59it/s]

...

Accuracy of the network on the test data: 27 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 15 %
Accuracy of activity  1 : 84 %
Accuracy of activity  2 :  3 %
Accuracy of activity  3 :  4 %


 94%|█████████▍| 9003/9600 [04:18<00:16, 35.28it/s]

Accuracy of the network on the test data: 26 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 14 %
Accuracy of activity  1 : 83 %
Accuracy of activity  2 :  4 %
Accuracy of activity  3 :  2 %


100%|██████████| 9600/9600 [04:34<00:00, 34.93it/s]

Accuracy of the network on the test data: 26 %
[498.0, 503.0, 488.0, 511.0]
Accuracy of activity  0 : 10 %
Accuracy of activity  1 : 80 %
Accuracy of activity  2 :  6 %
Accuracy of activity  3 :  6 %


In [18]:
test = [1, 0, 3]
torch.tensor(test).unsqueeze(axis=-1)
(torch.tensor([0]), torch.tensor([2]), torch.tensor([1])) in [(0,1,1)]

False